# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=3987156049
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [4]:
sgmt

,Name,Time,Speed,Power,Info
0,regional Envia-Salida Bogotá 18.36km 121m -1%,36:00,30.6km/h,276W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."
1,Mayorca-Aguacatala 4.58km 49m -1%,9:20,29.5km/h,240W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."
2,Mayorca - Pandequeso 1.82km 19m -1%,4:22,25.1km/h,192W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."
3,Mayorca - Bancolombia 8.17km 71m -1%,15:56,30.8km/h,260W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."
4,Av. Regional: Puente Pandequeso - Puente de la...,8:58,32.6km/h,280W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."
5,Av. Regional: Puente Pandequeso - Puente de la...,5:02,32.9km/h,284W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."
6,Carrefour Vegas- Puente de la 30 4.89km 37m ...,8:53,33.1km/h,296W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."
7,Retorno Ayura -Pte 12 sur 0.84km 7m -1%,1:29,34.3km/h,311W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."
8,12 sur (Aguacatala) - La 4 sur 0.98km 13m -1%,1:46,33.5km/h,300W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."
9,12 sur (Aguacatala) - La 10 2.09km 19m -1%,3:51,32.7km/h,273W,"{'Calories': {'Avg': '2452', 'Max': '2452'}, '..."


In [5]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [6]:
df.to_json(db_file)

### Save notebook here

In [7]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [8]:
!bash commit.sh

[master bae6afb] Morning Ride 5:35 AM on Sunday, September 13, 2020
 2 files changed, 323 insertions(+), 11 deletions(-)
 rewrite db/strava.json (74%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 18.68 KiB | 3.74 MiB/s, done.
Total 5 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:restrepo/strava.git
   3a9bed9..bae6afb  master -> master


In [9]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 
